<a href="https://colab.research.google.com/github/kiplangatkorir/multi-agent-supernet/blob/main/multi_agent_supernet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import random

class Agent:
    def __init__(self, name, capability, cost):
        self.name = name
        self.capability = capability
        self.cost = cost  # Just represents token/API cost

    def execute(self, task):
        """ Executes a task based on capability and complexity """
        complexity = task["complexity"]
        success_prob = min(1, self.capability / complexity)
        return np.random.rand() < success_prob

class AgenticSupernet:
    def __init__(self, agents, entropy_weight=0.1):
        self.agents = agents
        self.architecture_distribution = np.ones(len(agents)) / len(agents)
        self.entropy_weight = entropy_weight

    def update_distribution(self, agent_idx, reward):
        """ Reward updates with entropy regularization """
        self.architecture_distribution[agent_idx] += reward
        self.architecture_distribution /= self.architecture_distribution.sum()  # Normalize

        entropy = -np.sum(self.architecture_distribution * np.log(self.architecture_distribution + 1e-8))
        self.architecture_distribution += self.entropy_weight * entropy
        self.architecture_distribution /= self.architecture_distribution.sum()  # Renormalize

    def sample_architecture(self, task, num_samples=3):
        """ Dynamically selects agents with cost constraints """
        sampled_agents = np.random.choice(
            self.agents, size=num_samples, p=self.architecture_distribution, replace=True
        )

        # Filter based on capability-cost ratio
        selected_agents = []
        for agent in sampled_agents:
            if agent.capability >= task["complexity"] * random.uniform(0.5, 1.5):
                selected_agents.append(agent)

        if task["complexity"] <= 3:
            selected_agents = [agent for agent in selected_agents if agent.cost < 5]

        return selected_agents if selected_agents else [random.choice(self.agents)]

class Controller:
    def __init__(self, supernet):
        self.supernet = supernet

    def allocate_agents(self, task):
        return self.supernet.sample_architecture(task)

    def execute_task(self, task):
        agents = self.allocate_agents(task)
        print(f"Executing task: {task['name']} with {', '.join(a.name for a in agents)}")

        success = any(agent.execute(task) for agent in agents)
        for agent in agents:
            agent_idx = self.supernet.agents.index(agent)
            reward = 0.1 if success else -0.05
            self.supernet.update_distribution(agent_idx, reward)

        print(f"Task {task['name']} {'succeeded' if success else 'failed'}.\n")
        return success

# Define agents with costs
agents = [
    Agent("BasicAgent", capability=1, cost=1),
    Agent("MidAgent", capability=5, cost=3),
    Agent("ExpertAgent", capability=10, cost=8),
]

# Initialize system
supernet = AgenticSupernet(agents)
controller = Controller(supernet)

# Example tasks
tasks = [
    {"name": "Simple Arithmetic", "complexity": 1},
    {"name": "Web Navigation", "complexity": 5},
    {"name": "Advanced Code Generation", "complexity": 10},
]

# Run tasks dynamically
for _ in range(5):
    for task in tasks:
        controller.execute_task(task)

# Final probability distribution
print("Final Agent Selection Probabilities:", supernet.architecture_distribution)


Executing task: Simple Arithmetic with MidAgent, MidAgent
Task Simple Arithmetic succeeded.

Executing task: Web Navigation with ExpertAgent
Task Web Navigation succeeded.

Executing task: Advanced Code Generation with ExpertAgent, ExpertAgent
Task Advanced Code Generation succeeded.

Executing task: Simple Arithmetic with MidAgent, MidAgent, BasicAgent
Task Simple Arithmetic succeeded.

Executing task: Web Navigation with ExpertAgent, MidAgent
Task Web Navigation succeeded.

Executing task: Advanced Code Generation with ExpertAgent
Task Advanced Code Generation succeeded.

Executing task: Simple Arithmetic with MidAgent, MidAgent
Task Simple Arithmetic succeeded.

Executing task: Web Navigation with MidAgent, MidAgent
Task Web Navigation succeeded.

Executing task: Advanced Code Generation with BasicAgent
Task Advanced Code Generation failed.

Executing task: Simple Arithmetic with MidAgent, MidAgent
Task Simple Arithmetic succeeded.

Executing task: Web Navigation with MidAgent, MidA